In [ ]:
import sys
sys.path.append('..')
from src.transforms import instantiate_datamodule_transforms
from src.utils import init_config
from src.datasets.synthetic_config import CLASS_NAMES, CLASS_COLORS, STUFF_CLASSES

cfg = init_config(overrides=[
    f"experiment=semantic/synthetic_11g",
    f"datamodule.load_full_res_idx=True"
])
transforms_dict = instantiate_datamodule_transforms(cfg.datamodule)

In [ ]:
from src.transforms import *
from src.datasets.synthetic import read_synthetic

filepath = '../data/ontras/ontras_3.las'

data = read_synthetic(filepath)
data = SampleXYTiling(x=1, y=1, tiling=3)(data)

In [ ]:
import seaborn as sns

data = GroundElevation(z_threshold=5, scale=20)(data)
sns.displot(data.elevation)
data.show(class_names=CLASS_NAMES, class_colors=CLASS_COLORS, keys='elevation')

In [ ]:
data = KNN(k=25, r_max=2)(data)
data = PointFeatures(keys=('elevation'))(data)
# data = PointFeatures(keys=('hsv', 'linearity', 'planarity', 'scattering', 'verticality'))(data)
data.show(class_names=CLASS_NAMES, class_colors=CLASS_COLORS, keys=data.keys)

In [ ]:
data = AdjacencyGraph(k=10, w=1)(data)

# Copy desired features to `x`
data = AddKeysTo(keys=['linearity', 'planarity', 'scattering', 'elevation'], to='x', delete_after=False)(data)

# Compute the hierarchical partition
nag = CutPursuitPartition(
    regularization=[0.1, 0.2], 
    spatial_weight=[0.1, 0.01], 
    cutoff=[10, 30], 
    iterations=15, 
    k_adjacency=10)(data)